In [26]:
import numpy as np
from numpy.linalg import pinv
from time import time
import matplotlib.pyplot as plt


In [27]:
# lengths in m 
l0, l1, l2, l3 = 0.12225, 0.056, 0.177, 0.135952

# origin
x0, y0, z0 = 0, 0, 0

# Metric
metric = 0.01 

# Destination
#e_des = [-0.04127370851174543,-0.1709845251223143]
#e_l_des = [0.04127370851174543,-0.1709845251223143]
e_des_r = [-0.1,0]
e_des_l = [0.1,0]

# Initial theta
t_r = [0,0,0,0]
t_l = [0,0,0,0]

# Error
err_r = 10
threshold = 0.001
# Time period
dt = 0.001

dt0, dt1, dt2, dt3 = (dt,0,0,0), (0,dt,0,0), (0,0,dt,0), (0,0,0,dt)

# linearisation val
e_lin = 0.001


In [28]:
class ForwardKinematics():
    def get_right_end_eff(self,t=(0,0,0,0)):
        (t0,t1,t2,t3) = t
        x = np.cumsum([x0,0,0,0,0])
        y = np.cumsum([y0 , -(l0*np.cos(t0)) , -(l1*np.cos(t0 + t1)) ,  l2*np.sin(t0 + t1 + t2)   ,   l3*np.sin(t0 + t1 + t2 + t3)  ])
        z = np.cumsum([z0 ,  l0*np.sin(t0)   ,   l1*np.sin(t0 + t1)  , -(l2*np.cos(t0 + t1 + t2)) , -(l3*np.cos(t0 + t1 + t2 + t3)) ])
        #print("right posi",x,y,z,sep='\n', end='\n\n')
        return list(x),list(y),list(z) 

    def get_right_orientation(self,t=(0,0,0,0)):
        (t0,t1,t2,t3) = t
        xt = np.cumsum([t0,  t1,  t2-1.5708,  t3])
        yt = np.cumsum([ 0,  0 ,     0     ,   0])
        zt = np.cumsum([ 0,  0 ,     0     ,   0])
        #print("right orient",xt,yt,zt,sep='\n', end='\n\n')
        return list(xt),list(yt),list(zt) 
    
    def get_left_end_eff(self,t=(0,0,0,0)):
        (t0,t1,t2,t3) = t
        x = np.cumsum([x0,0,0,0,0])
        y = np.cumsum([y0, l0*np.cos(t0) , l1*np.cos(t0 + t1) ,  l2*np.sin(t0 + t1 + t2) ,  l3*np.sin(t0 + t1 + t2 + t3) ])
        z = np.cumsum([z0, l0*np.sin(t0) , l1*np.sin(t0 + t1) , -l2*np.cos(t0 + t1 + t2) , -l3*np.cos(t0 + t1 + t2 + t3) ])
        #print("left posi",x,y,z,sep='\n',end='\n\n')
        return list(x),list(y),list(z)    

    def get_left_orientation(self,t=(0,0,0,0)):
        (t0,t1,t2,t3) = t
        xt = np.cumsum([t0,  t1,  t2-1.5708,  t3])
        yt = np.cumsum([ 0,  0 ,     0     ,   0])
        zt = np.cumsum([ 0,  0 ,     0     ,   0])
        #print("left orient",xt,yt,zt,sep='\n', end='\n\n')

fk = ForwardKinematics()

plt.figure()
plt.ion()

<Figure size 432x288 with 0 Axes>

In [11]:
class InverseKinematics(ForwardKinematics):
    def __init__(self, t=(0,0,0,0)):
        self.t = t
            
    def de_dt(self,d_t):
        t = self.t
        #print(d_t)
        end_eff_right_t = ForwardKinematics.get_right_end_eff(t)
        #print(end_eff_right_t)
        end_eff_right_dt = ForwardKinematics.get_right_end_eff(d_t)
        #print(end_eff_right_dt)
        # Right arm del_e
        del_e_x,del_e_y,del_e_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_right_dt,end_eff_right_t)]
        # Jacobian elements for right arm
        del_e_dt = [list(map(lambda x: x/dt, list(del_e_x))),list(map(lambda x: x/dt, list(del_e_y))),list(map(lambda x: x/dt, list(del_e_z)))]
        return del_e_dt
            
    def jacobian(self,t):
        pass
    
    
    def step(self,dt):
        pass
    

In [6]:
ik = InverseKinematics()
t = [0,0,0,0]

print(ik.de_dt(t0))
print(ik.de_dt(t1))
print(ik.de_dt(t2))
print(ik.de_dt(t3))



NameError: name 't0' is not defined

In [ ]:

while not (err_r < threshold): #if True:

    tr0 = list(map(lambda x1,x2: x1+x2,t_r,dt0))
    tr1 = list(map(lambda x1,x2: x1+x2,t_r,dt1))
    tr2 = list(map(lambda x1,x2: x1+x2,t_r,dt2))
    tr3 = list(map(lambda x1,x2: x1+x2,t_r,dt3))

    # Right arm parameters
    # Right arm e'(right e = er)
    end_eff_right_t = fk.get_right_end_eff(t_r)
    #print(end_eff_right_t)
    end_eff_right_t0 = fk.get_right_end_eff(tr0)
    end_eff_right_t1 = fk.get_right_end_eff(tr1)
    end_eff_right_t2 = fk.get_right_end_eff(tr2)
    end_eff_right_t3 = fk.get_right_end_eff(tr3)

    # Right arm del_e
    del_r_t0_x,del_r_t0_y,del_r_t0_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_right_t0,end_eff_right_t)]
    del_r_t1_x,del_r_t1_y,del_r_t1_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_right_t1,end_eff_right_t)]
    del_r_t2_x,del_r_t2_y,del_r_t2_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_right_t2,end_eff_right_t)]
    del_r_t3_x,del_r_t3_y,del_r_t3_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_right_t3,end_eff_right_t)]
    
    # Jacobian elements for right arm
    del_er_dt0 = [list(map(lambda x: x/dt, list(del_r_t0_x))),list(map(lambda x: x/dt, list(del_r_t0_y))),list(map(lambda x: x/dt, list(del_r_t0_z)))]
    del_er_dt1 = [list(map(lambda x: x/dt, list(del_r_t1_x))),list(map(lambda x: x/dt, list(del_r_t1_y))),list(map(lambda x: x/dt, list(del_r_t1_z)))]
    del_er_dt2 = [list(map(lambda x: x/dt, list(del_r_t2_x))),list(map(lambda x: x/dt, list(del_r_t2_y))),list(map(lambda x: x/dt, list(del_r_t2_z)))]
    del_er_dt3 = [list(map(lambda x: x/dt, list(del_r_t3_x))),list(map(lambda x: x/dt, list(del_r_t3_y))),list(map(lambda x: x/dt, list(del_r_t3_z)))]

    # Right arm Jacobian
    j_r = [[del_er_dt0[1][4] , del_er_dt1[1][4] , del_er_dt2[1][4] , del_er_dt3[1][4]],
           [del_er_dt0[2][4] , del_er_dt1[2][4] , del_er_dt2[2][4] , del_er_dt3[2][4]]]
    
    # Right arm inverse Jacobian
    J_r_inv = pinv(j_r)

    er = [end_eff_right_t[1][4],end_eff_right_t[2][4]]
    diff_r = list(np.array(e_des_r) - np.array(er))
    de_lin_r = list(np.sign(diff_r)*e_lin)
    dt_new_r = np.dot(J_r_inv,de_lin_r)
    
    t_r = list(map(lambda x1,x2: x1+x2,t_r,dt_new_r))
    
    pos_cur_right = fk.get_right_end_eff(t_r)
    cur_end_eff_pos_r = [pos_cur_right[1][4],pos_cur_right[2][4]]
    err_r = np.sqrt(np.sum(((np.array(cur_end_eff_pos_r)-np.array(e_des_r))**2),axis=0))
    (_,yr,zr) = pos_cur_right

    # Left arm parameters
    # Left arm e'(right e = er)
    tl0 = list(map(lambda x1,x2: x1+x2,t_l,dt0))
    tl1 = list(map(lambda x1,x2: x1+x2,t_l,dt1))
    tl2 = list(map(lambda x1,x2: x1+x2,t_l,dt2))
    tl3 = list(map(lambda x1,x2: x1+x2,t_l,dt3))
    
    end_eff_left_t = fk.get_right_end_eff(t_l)
    #print(end_eff_right_t)
    end_eff_left_t0 = fk.get_left_end_eff(tl0)
    end_eff_left_t1 = fk.get_left_end_eff(tl1)
    end_eff_left_t2 = fk.get_left_end_eff(tl2)
    end_eff_left_t3 = fk.get_left_end_eff(tl3)

    # Right arm del_e
    del_l_t0_x,del_l_t0_y,del_l_t0_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_left_t0,end_eff_left_t)]
    del_l_t1_x,del_l_t1_y,del_l_t1_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_left_t1,end_eff_left_t)]
    del_l_t2_x,del_l_t2_y,del_l_t2_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_left_t2,end_eff_left_t)]
    del_l_t3_x,del_l_t3_y,del_l_t3_z = [map(lambda x1,x2: x1-x2,i,j) for i,j in zip(end_eff_left_t3,end_eff_left_t)]
    
    # Jacobian elements for right arm
    del_el_dt0 = [list(map(lambda x: x/dt, list(del_l_t0_x))),list(map(lambda x: x/dt, list(del_l_t0_y))),list(map(lambda x: x/dt, list(del_l_t0_z)))]
    del_el_dt1 = [list(map(lambda x: x/dt, list(del_l_t1_x))),list(map(lambda x: x/dt, list(del_l_t1_y))),list(map(lambda x: x/dt, list(del_l_t1_z)))]
    del_el_dt2 = [list(map(lambda x: x/dt, list(del_l_t2_x))),list(map(lambda x: x/dt, list(del_l_t2_y))),list(map(lambda x: x/dt, list(del_l_t2_z)))]
    del_el_dt3 = [list(map(lambda x: x/dt, list(del_l_t3_x))),list(map(lambda x: x/dt, list(del_l_t3_y))),list(map(lambda x: x/dt, list(del_l_t3_z)))]

    # Right arm Jacobian
    j_l = [[del_el_dt0[1][4] , del_el_dt1[1][4] , del_el_dt2[1][4] , del_el_dt3[1][4]],
           [del_el_dt0[2][4] , del_el_dt1[2][4] , del_el_dt2[2][4] , del_el_dt3[2][4]]]
    
    # Right arm inverse Jacobian
    J_l_inv = pinv(j_l)

    el = [end_eff_left_t[1][4],end_eff_left_t[2][4]]
    diff_l = list(np.array(e_des_l) - np.array(el))
    de_lin_l = list(np.sign(diff_l)*e_lin)
    dt_new_l = np.dot(J_l_inv,de_lin_l)
    
    t_l = list(map(lambda x1,x2: x1+x2,t_l,dt_new_l))
    
    pos_cur_left = fk.get_left_end_eff(t_l)
    cur_end_eff_pos_l = [pos_cur_left[1][4],pos_cur_left[2][4]]
    err_l = np.sqrt(np.sum(((np.array(cur_end_eff_pos_l)-np.array(e_des_l))**2),axis=0))
    (_,yl,zl) = pos_cur_left

    plt.clf()
    #rline, = ax.plot([y], [z], 'o-', lw=2)
    plt.plot(yr,zr, 'o-', lw=2)
    plt.plot(yl,zl, 'o-', lw=2)
    plt.plot(e_des_r[0],e_des_r[1], 'o')
    plt.plot(e_des_l[0],e_des_l[1], 'o')
    plt.grid(True)
    plt.axis((-0.6, 0.6, -0.6, 0.6))
    plt.pause(0.01)
    #print(err_r,err_l)
    print(t_r,t_l)
    print(cur_end_eff_pos_r)
    print(cur_end_eff_pos_l)
    print("--------------")

plt.pause(10)


In [86]:
a=[1,2,3,4]
b = list(-np.array(a))
c = [-2,-4,1,2]

In [89]:
np.dot(a,b)

-30

In [83]:
a = np.array(a)
b = np.array(b)
c = np.array(c)

In [84]:
np.maximum(np.minimum(c,a),b)

array([-1, -2,  1,  2])

In [79]:
min(a,c)

[-2, -4, 10, 20]

array([ -1,  -4,  -9, -16])

In [ ]:
ma

In [33]:
min(max(np.array(a)*b,-b),b)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
while not (err_r < threshold):
    

In [51]:
max(np.array(a)*b,list(b))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [46]:
-np.array(a)


array([-2, -3])